# Mixed Integer Programming(MIP) for the instance  'Meal Planning for the New Millennium'(MnM)

## Download the library 
First install [DOcplex](https://cdn.rawgit.com/IBMDecisionOptimization/docplex-doc/2.0.15/docs/index.html) Python library if needed. Consider the scale for this problem, [CPLEX](https://www.ibm.com/analytics/cplex-optimizer) or [DOcplexcloud](https://developer.ibm.com/docloud) is needed. 

In [1]:
import sys
try:
    import docplex.mp
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install docplex
    else:
        !pip install --user docplex

Subscribe to IBM Decision Optimization on Cloud service service if you do not want to use a local solver.
Get the service URL and your personal API key and enter your credentials here if accurate:

In [2]:
url = "https://api-oaas.docloud.ibmcloud.com/job_manager/rest/v1/"
key = "api_cbbbfd41-bb59-44b1-a2c5-44dc754e8255"

## Create a model instance

In [3]:
from docplex.mp.model import Model

MnM = Model(name='Meal Planning for the New Millennium')

The model for this instance is describe below

\begin{align*}
Indices: & \\
& Person: i \in {\{1,2\}} \\
& Day: t \in {\{1,...,5\}} \\
& Recipes: k \in {\{1,...,K\}} \\
Parameters: & \\
& Available \ Time: T_{it} \\
& Budget: B \\
& Recipe \ price: \pi_k  \\
& Nutrition \ Vector: A_k=(c_k,f_k,p_k,s_k,car_k)^T \\
& Minimum \ Nutrition \ Requirements: R_l=(c_l,f_l,p_l,s_l,car_l)^T \\
& Maximum \ Nutrition \ Requirements: R_u=(c_u,f_u,p_u,s_u,car_u)^T \\
& Cooking \ Time: \tau_k \\
& Ratings: r_{ik} \\
& Filtering: F \text{ indicate the indices for recipes including allergy or dislike food} \\
Variables: & \\
& x_{itk} \in {\{0,1\}}  \text{ indicate whether person i cook recipe k on day t} \\
& w: inequity \\
\\
\\
\max & \sum_{j=1}^{K}\sum_{t=1}^5\sum_{i=1}^2 (r_{1k}+r_{2k}) x_{itk} -\alpha w 
\end{align*}
subject to
\begin{align*}
\sum_{j=1}^{K}\sum_{t=1}^5\sum_{i=1}^2  \pi_k x_{itk} & \leq B \\
\sum_{j=1}^{K} \tau_k x_{itk} & \leq T_{it}, \forall i \in {\{1,2\}},t \in {\{1,...,5\}} \\
\sum_{j=1}^{K}\sum_{t=1}^5 \tau_k (x_{1tk}-x_{2tk}) & \geq -w \\
\sum_{j=1}^{K}\sum_{t=1}^5 \tau_k (x_{1tk}-x_{2tk}) & \leq w \\
\sum_{j=1}^{K}\sum_{t=1}^5\sum_{i=1}^2 A_k x_{itk} & \geq R_l \\
\sum_{j=1}^{K}\sum_{t=1}^5\sum_{i=1}^2 A_k x_{itk} & \leq R_u \\
\sum_{k=1}^K\sum_{i=1}^2  x_{itk} & = 1, \forall t \in {\{1,...,5\}} \\
\sum_{t=1}^5\sum_{i=1}^2  x_{itk} & \leq 1, \forall k \in {\{1,...,K\}} \\
x_{itk} & = 0, \forall i \in {\{1,2\}},t \in {\{1,...,5\}},k \in F\\
x_{itk} & \in\{{0,1}\}, \forall i \in {\{1,2\}},t \in {\{1,...,5\}},k \in {\{1,...,K\}} 
\end{align*}

In [4]:
import numpy as np
import pandas as pd

In [5]:
rating_matrix = pd.read_csv('recipe-user-rating-after-matrix-completion.csv', index_col=False)

In [6]:
rating_matrix.head()

,X,X20.Minute.Pesto.Chicken.Pizza,X30.Minute.Sheet.Pan.Chicken.Caprese,X4.Ingredient.BBQ.Baked.Salmon,X5.Ingredient.Cheesecake,X5.Ingredient.Honey.BBQ.Baked.Chicken.Wings,X5.Ingredient.Pound.Cake,Amazing.Rosemary.Sweet.Potato.Fries,Apple.Cinnamon.Muffins,Apple.Pie.Baked.Oatmeal,...,Sweet.and.Salty.Candied.Bacon,Sweet.and.Spicy.Pecans,Twice.Baked.Potatoes,Vegan.Baked.Ziti,Vegan.Banana.Bread,Vegan.Black.Bean.Enchiladas,Vegan.Chickpea..Meatloaf.,Vegan.Spinach.Lasagna,Vegetarian.Bean.and.Cheese.Taco.Casserole,Za.atar.Roasted.Cauliflower.Steaks.with.Bean.Salad
0,0,5.000000,4.682655,4.766987,5.000000,5.0,3.594327,4.466223,4.556360,4.820759,...,4.211138,4.679252,5.000000,5.679800,5.124751,5.057999,5.000000,5.000000,5.873637,2.978791
1,1,4.000000,4.000000,5.000000,5.000000,5.0,3.000000,4.000000,5.000000,4.000000,...,1.000000,4.312266,5.000000,5.000000,4.701904,5.000000,4.125224,4.281432,5.324161,2.789098
2,2,4.707114,4.407471,5.000000,4.586997,4.0,1.000000,4.211873,4.289544,4.458705,...,4.000000,4.440338,5.000000,5.000000,5.000000,5.000000,4.169489,4.394664,5.382821,2.921127
3,3,5.000000,5.000000,5.000000,4.870668,5.0,5.000000,4.497715,4.577213,4.720207,...,5.000000,4.754628,4.506734,5.376022,5.138163,5.000000,4.394521,4.693250,5.674733,3.171949
4,4,4.500000,4.500000,4.738222,5.000000,5.0,3.740481,4.515841,4.591708,4.692075,...,5.000000,4.788643,5.000000,5.000000,5.150905,5.060170,4.345750,4.000000,5.613377,4.000000


In [7]:
rating_matrix.shape

(35, 126)

## Rearrange the recipes-user-rating matrix

In [8]:
columns =  list(pd.read_csv('final_rating_data.csv', index_col=False).columns)

In [9]:
rating_matrix.columns = columns

In [10]:
rating_matrix.head()

,Unnamed: 0,20-Minute Pesto Chicken Pizza,30-Minute Sheet Pan Chicken Caprese,4-Ingredient BBQ Baked Salmon,5-Ingredient Cheesecake,5-Ingredient Honey-BBQ Baked Chicken Wings,5-Ingredient Pound Cake,Amazing Rosemary Sweet Potato Fries,Apple Cinnamon Muffins,Apple Pie Baked Oatmeal,...,Sweet and Salty Candied Bacon,Sweet and Spicy Pecans,Twice Baked Potatoes,Vegan Baked Ziti,Vegan Banana Bread,Vegan Black Bean Enchiladas,"Vegan Chickpea ""Meatloaf""",Vegan Spinach Lasagna,Vegetarian Bean and Cheese Taco Casserole,Za'atar-Roasted Cauliflower Steaks with Bean Salad
0,0,5.000000,4.682655,4.766987,5.000000,5.0,3.594327,4.466223,4.556360,4.820759,...,4.211138,4.679252,5.000000,5.679800,5.124751,5.057999,5.000000,5.000000,5.873637,2.978791
1,1,4.000000,4.000000,5.000000,5.000000,5.0,3.000000,4.000000,5.000000,4.000000,...,1.000000,4.312266,5.000000,5.000000,4.701904,5.000000,4.125224,4.281432,5.324161,2.789098
2,2,4.707114,4.407471,5.000000,4.586997,4.0,1.000000,4.211873,4.289544,4.458705,...,4.000000,4.440338,5.000000,5.000000,5.000000,5.000000,4.169489,4.394664,5.382821,2.921127
3,3,5.000000,5.000000,5.000000,4.870668,5.0,5.000000,4.497715,4.577213,4.720207,...,5.000000,4.754628,4.506734,5.376022,5.138163,5.000000,4.394521,4.693250,5.674733,3.171949
4,4,4.500000,4.500000,4.738222,5.000000,5.0,3.740481,4.515841,4.591708,4.692075,...,5.000000,4.788643,5.000000,5.000000,5.150905,5.060170,4.345750,4.000000,5.613377,4.000000


In [11]:
recipes = pd.read_csv('recipe_info.csv', index_col=False)

In [12]:
recipes.head()

,Unnamed: 0,title,rating,time,price,calories,sodium,fat,protein,carbs,personal_rating,ingredients,url
0,1,Four-Cheese Baked Spaghetti,4.5,75.0,38.35,570.0,1230.0,28.0,31,51,"[['Kaitlin Judd', 3.0], ['Em Spreadborough', 5...","['spaghetti ', 'lowmoisture ricotta cheese ', ...",https://www.yummly.com/recipe/Four-Cheese-Bake...
1,3,Creamy Spinach Stuffed Mushrooms,4.5,33.0,17.99,70.0,180.0,5.0,3,3,"[['Avery', 1.0], ['Ischa Bremer', 4.0], ['Kiau...","['cremini mushrooms ', 'cream cheese ', 'grate...",https://www.yummly.com/recipe/Creamy-Spinach-S...
2,7,Best Ever Chocolate Chip Muffins,4.5,41.0,30.70,270.0,270.0,14.0,4,34,"[['Sophia', 5.0], ['Heidi Crosgrove-Trobaugh',...","['unsalted butter ', 'granulated sugar ', 'lig...",https://www.yummly.com/recipe/Best-Ever-Chocol...
3,8,Sheet Pan Yellow Squash and Chicken Sausage,4.5,42.0,20.08,340.0,960.0,24.0,22,10,"[['Holly Lynch', 5.0], ['Amy J.', 5.0], ['Cath...","['yellow summer squash ', 'cooked chicken saus...",https://www.yummly.com/recipe/Sheet-Pan-Yellow...
4,9,Hearty Italian Beef and Vegetable Soup,4.5,40.0,23.10,240.0,920.0,7.0,27,19,"[['Isabella', 5.0], ['Rejeana Black', 5.0], ['...","['medium onion ', 'garlic cloves ', 'large car...",https://www.yummly.com/recipe/Hearty-Italian-B...


In [13]:
title = recipes.title

In [14]:
rating_matrix = rating_matrix[title]

In [15]:
rating_matrix.head()

,Four-Cheese Baked Spaghetti,Creamy Spinach Stuffed Mushrooms,Best Ever Chocolate Chip Muffins,Sheet Pan Yellow Squash and Chicken Sausage,Hearty Italian Beef and Vegetable Soup,Loaded Baked Potatoes,Soft and Chewy Keto “Sugar” Cookies,Amazing Rosemary Sweet Potato Fries,Farmers' Market Vegetable Bake with Sharp Cheddar and Parmesan,Oven Baked BBQ Baby Back Ribs,...,Easy Sweet Potatoes Au Gratin,Savory Keto Breakfast Cookies,Sweet and Salty Candied Bacon,Sheet Pan Honey Mustard Pork Chops with Roasted Veggies,Vegan Spinach Lasagna,Italian Sausage-Stuffed Mushrooms,Vegetarian Bean and Cheese Taco Casserole,Easy Honey-Mustard Chicken Thighs,Garlic Roasted Broccoli,Crispy Panko-Parmesan Chicken Breasts
0,5.000000,3.687671,3.333951,5.913462,4.276111,4.307912,3.000000,4.466223,4.549404,4.903512,...,5.000000,4.987605,4.211138,4.000000,5.000000,5.000000,5.873637,5.000000,5.000000,5.000000
1,4.588172,3.423826,3.120542,5.000000,3.950582,3.969670,5.000000,4.000000,4.101115,5.000000,...,4.677771,4.560406,1.000000,5.000000,4.281432,3.919499,5.324161,5.000000,5.000000,4.000000
2,5.000000,1.000000,3.267050,5.313481,4.078932,4.087140,4.305855,4.211873,4.120461,4.617607,...,4.789638,4.655498,4.000000,3.000000,4.394664,4.005509,5.382821,4.000000,5.000000,4.197244
3,4.999481,5.000000,3.546508,5.550495,4.377511,4.376049,4.000000,4.497715,5.000000,4.927861,...,5.104416,4.948878,5.000000,4.076175,4.693250,4.261797,5.674733,4.658993,4.898368,5.000000
4,5.000000,3.887457,3.627057,5.431040,4.420124,4.406919,5.000000,4.515841,4.246234,4.944136,...,5.113198,4.942924,5.000000,4.073033,4.000000,4.261130,5.613377,4.667516,4.893291,4.000000


## Ingredients Filter

In [16]:
allergy = "salmon"
picky = "peanut" 
desserts = ["muffins","cookies","cake","brownies","chocolate","fries", "bread", "toast"]

In [17]:
matching = recipes[recipes.ingredients.str.contains(allergy, case=False) | recipes.ingredients.str.contains(picky, case=False)]

In [18]:
for dessert in desserts:
    matching = pd.concat([recipes[recipes.title.str.contains(dessert, case=False)],matching])

In [19]:
matching.sort_index(inplace=True)

In [20]:
matching.head()

,Unnamed: 0,title,rating,time,price,calories,sodium,fat,protein,carbs,personal_rating,ingredients,url
2,7,Best Ever Chocolate Chip Muffins,4.5,41.0,30.70,270.0,270.0,14.0,4,34,"[['Sophia', 5.0], ['Heidi Crosgrove-Trobaugh',...","['unsalted butter ', 'granulated sugar ', 'lig...",https://www.yummly.com/recipe/Best-Ever-Chocol...
2,7,Best Ever Chocolate Chip Muffins,4.5,41.0,30.70,270.0,270.0,14.0,4,34,"[['Sophia', 5.0], ['Heidi Crosgrove-Trobaugh',...","['unsalted butter ', 'granulated sugar ', 'lig...",https://www.yummly.com/recipe/Best-Ever-Chocol...
6,14,Soft and Chewy Keto “Sugar” Cookies,4.5,23.0,22.04,240.0,180.0,23.0,6,6,"[['Tamara Canion', 5.0], ['Jason T.', 3.0], ['...","['unsalted butter ', 'full fat cream cheese ',...",https://www.yummly.com/recipe/Soft-and-Chewy-K...
7,17,Amazing Rosemary Sweet Potato Fries,4.5,30.0,5.44,130.0,360.0,3.5,3,24,"[['Yusuf', 5.0], ['Elaine', 5.0], ['Nick', 5.0...","['sweet potatoes ', 'olive oil ', 'salt ', 'bl...",https://www.yummly.com/recipe/Amazing-Rosemary...
11,22,Healthy Salmon & Veggie Sheet Pan Dinner,4.5,40.0,33.46,500.0,270.0,11.0,40,69,"[['Abigail Clark', 4.0], ['Tyler Binkley', 5.0...","['salmon ', 'baby potatoes ', 'asparagus ', 's...",https://www.yummly.com/recipe/Healthy-Salmon-_...


In [21]:
filtering = list(matching.index)

## Define the variables

In [22]:
# x_{itk} are binary variable
x = {(i,t,k): MnM.binary_var(name='x_{0}_{1}_{2}'.format(i,t,k)) for i in range(2) for t in range(5) for k in range(recipes.shape[0])}

In [23]:
# time inequality
w = MnM.continuous_var(name='w', lb=0)

## Define the parameters

In [24]:
# recipe matrix 
K = recipes.shape[0] #number of recipes
T = [[200,50,100,50,200],[0,100,100,100,100]]

#rating for user 0 and user 1
rating = rating_matrix.iloc[0:2, -K:].values.tolist()

calories = recipes.calories
protein = recipes.protein
fat = recipes.fat
sodium = recipes.sodium
carbs = recipes.carbs

price = recipes.price

tau = recipes.time

# lower(1) and upper(2) bound of nutrient
c_bound=[1200, 3000]
p_bound= [25 ,88]
f_bound=[22 , 39]
s_bound= [2200 ,2400]
car_bound= [112, 163]
B = 120 #budget

# parameter for objective
alpha = 0.1

## Define the constraints

In [25]:
# five meals constraints, one meal per day
for t in range(5):
    MnM.add_constraint(MnM.sum(x[i,t,k] for i in range(2) for k in range(K)) == 1, ctname = 'subject to five_meals_total')


# nutrition lower bound
MnM.add_constraint(MnM.sum(calories[k]*x[i,t,k] for i in range(2) for t in range(5) for k in range(K)) >= c_bound[0])
MnM.add_constraint(MnM.sum(protein[k]*x[i,t,k] for i in range(2) for t in range(5) for k in range(K)) >= p_bound[0])
MnM.add_constraint(MnM.sum(fat[k]*x[i,t,k] for i in range(2) for t in range(5) for k in range(K)) >= f_bound[0])
MnM.add_constraint(MnM.sum(sodium[k]*x[i,t,k] for i in range(2) for t in range(5) for k in range(K)) >= s_bound[0])
MnM.add_constraint(MnM.sum(carbs[k]*x[i,t,k] for i in range(2) for t in range(5) for k in range(K)) >= car_bound[0])

# nutrition upper bound
MnM.add_constraint(MnM.sum(calories[k]*x[i,t,k] for i in range(2) for t in range(5) for k in range(K)) <= c_bound[1] )
MnM.add_constraint(MnM.sum(protein[k]*x[i,t,k] for i in range(2) for t in range(5) for k in range(K)) <= p_bound[1] )
MnM.add_constraint(MnM.sum(fat[k]*x[i,t,k] for i in range(2) for t in range(5) for k in range(K)) <= f_bound[1] )
MnM.add_constraint(MnM.sum(sodium[k]*x[i,t,k] for i in range(2) for t in range(5) for k in range(K)) <= s_bound[1] )
MnM.add_constraint(MnM.sum(carbs[k]*x[i,t,k] for i in range(2) for t in range(5) for k in range(K)) <= s_bound[1] )

#budget constraint
MnM.add_constraint(MnM.sum(price[k]*x[i,t,k] for i in range(2) for t in range(5) for k in range(K)) <= B  )

        
# schedule time inequalities
MnM.add_constraint(MnM.sum(tau[k]* (x[0,t,k] - x[1,t,k]) for t in range(5) for k in range(K)) <= w)
MnM.add_constraint(MnM.sum(tau[k]* (x[0,t,k] - x[1,t,k]) for t in range(5) for k in range(K)) >= -w)

        
# schedule date constraint
for i in range(2):
    for t in range(5):
        MnM.add_constraint(MnM.sum(x[i,t,k] * tau[k] for k in range(K)) <= T[i][t])  
        
# Diversity constraint
for k in range(K):
    MnM.add_constraint(MnM.sum(x[i,t,k] for i in range(2) for t in range(5)) <= 1)

# Ingredients Filter
for i in range(2):
    for t in range(5):
        for k in filtering:
            MnM.add_constraint(x[i,t,k] == 0)

## Define the objective function

In [26]:
MnM.maximize(MnM.sum((rating[0][k]+rating[1][k])*x[i,t,k] for i in range(2) for t in range(5) for k in range(K)) - alpha*w)

## Solve the problem

In [27]:
MnM.print_information()

Model: Meal Planning for the New Millennium
 - number of variables: 1251
   - binary=1250, integer=0, continuous=1
 - number of constraints: 563
   - linear=563
 - parameters: defaults
 - problem type is: MILP


In [28]:
MnMs= MnM.solve(url=url, key=key, log_output=True)
assert MnMs
MnM.print_solution()

[2020-05-08T23:57:20Z, INFO] CPLEX version 12090000
[2020-05-08T23:57:20Z, INFO] Parameter file:
[2020-05-08T23:57:20Z, INFO] # -- This content is generated by DOcplex
[2020-05-08T23:57:20Z, INFO] CPLEX Parameter File Version 12.10.0.0
[2020-05-08T23:57:20Z, INFO] # --- end of generated prm data ---
[2020-05-08T23:57:20Z, WARN] Warning:  CPLEX version found in file python_meal_planning_for_the_new_millennium.prm (12.10.0.0) does not match
[2020-05-08T23:57:20Z, WARN]           this CPLEX version (12.9.0.0).
[2020-05-08T23:57:20Z, WARN] Changed parameter CPX_PARAM_THREADS from 0 to 10
[2020-05-08T23:57:20Z, INFO] Param[1,067] = 10
[2020-05-08T23:57:20Z, INFO] Param[1,130] = utf-8
[2020-05-08T23:57:20Z, INFO] Param[1,132] = -1
[2020-05-08T23:57:20Z, INFO] CPXPARAM_Threads                                 10
[2020-05-08T23:57:20Z, INFO] CPXPARAM_Output_CloneLog                         -1
[2020-05-08T23:57:20Z, INFO] CPXPARAM_Read_APIEncoding                        "utf-8"
[2020-05-08T23:57

In [29]:
results = pd.DataFrame(columns=['person','day','title','url','price','calories','time'])

In [30]:
count = 0
for i in range(2):
    for t in range(5):
        for k in range(K):
            if MnM.solution.get_value(x[i,t,k]):
                results.loc[count] = [i+1,t+1,recipes.loc[k].title,recipes.loc[k].url,recipes.loc[k].price,recipes.loc[k].calories,recipes.loc[k].time]
                count+=1

In [31]:
pd.set_option('display.max_colwidth', None)
results

,person,day,title,url,price,calories,time
0,1,1,"Vegan Chickpea ""Meatloaf""",https://www.yummly.com/recipe/Vegan-Chickpea-_Meatloaf_-9083673,32.35,380.0,75.0
1,1,2,Easy Honey-Mustard Chicken Thighs,https://www.yummly.com/recipe/Easy-Honey-Mustard-Chicken-Thighs-9029460,19.60,310.0,30.0
2,1,3,Garlic Roasted Broccoli,https://www.yummly.com/recipe/Garlic-Roasted-Broccoli-2684142,7.47,130.0,30.0
3,2,4,Vegan Black Bean Enchiladas,https://www.yummly.com/recipe/Vegan-Black-Bean-Enchiladas-9116010,24.05,100.0,60.0
4,2,5,Vegan Baked Ziti,https://www.yummly.com/recipe/Vegan-Baked-Ziti-2684090,34.57,360.0,75.0
